# Unique Fire, Beetle, and Drought Disturbance Combinations in Western US Forests

This notebook identifies areas in the Western US affected by unique combinations of fire, beetle infestations, and drought over five-year periods. The goal is to track and quantify regions experiencing disturbances from two or more of these factors during the study period.

Key tasks include:
- Isolating areas impacted by fire, beetle, and drought disturbances.
- Calculating the total area affected by each unique disturbance combination (fire/beetle, beetle/drought, drought/fire) for each five-year period.
- Generating geospatial outputs and CSV files summarizing the total affected area for each combination.

The output provides insights into how multiple disturbance factors overlap and impact forest regions over time.


In [ ]:
# Import packages and make file paths
import os
from glob import glob
import earthpy as et
from os import path
import pandas as pd
import rasterio as rio
import gc

home = path.join(et.io.HOME, "GitHub", "visualize-forest-disturbance")

forest_folder = path.join(home, "forest-disturbance-stack", "western-conus")
combine_folder = path.join(home, "forest-disturbance-stack", "combined-western-conus")
fiveyear_all = path.join(home, "forest-disturbance-stack", "five-year-all")
unique_folder = path.join(home, "forest-disturbance-stack", "fire-year-unique")

fire_beetle_out = path.join(unique_folder, "fire-beetle")
fire_drought_out = path.join(unique_folder, "fire-dought")
drought_beetle_out = path.join(unique_folder, "drought-beetle")
fiveyear_beetle_folder = path.join(home, "forest-disturbance-stack", "five-year-beetle")
fiveyear_fire_folder = path.join(home, "forest-disturbance-stack", "five-year-fire")
fiveyear_drought_folder = path.join(home, "forest-disturbance-stack", "five-year-drought")
unique_out = path.join(home, "forest-disturbance-stack", "unique-all")

In [ ]:
# Ensure all directories exist
directories = [fire_beetle_out, fire_drought_out, drought_beetle_out, unique_folder, unique_out]

for di in directories:
    if not os.path.exists(di):
        os.makedirs(di)

In [ ]:
def unique_combos(years1, years2, combo_folder, combo_name, arr1, arr2, years3 = None, arr3=None):
    """
    Create a raster representing the unique disturbance combinations between two or three disturbance types 
    (e.g., fire, beetle, and drought) over a given period, saving the result as a new geospatial file.

    Parameters
    ----------
    years1 : list
        List of file paths to the first disturbance type (e.g., beetle) rasters over a period.
    years2 : list
        List of file paths to the second disturbance type (e.g., fire) rasters over a period.
    combo_folder : str
        Path to the folder where the output combined raster file will be saved.
    combo_name : str
        Name of the output file representing the combined disturbance types.
    arr1 : str
        File path to the aggregated raster file representing the total number of years affected by the first disturbance.
    arr2 : str
        File path to the aggregated raster file representing the total number of years affected by the second disturbance.
    years3 : list, optional
        List of file paths to the third disturbance type (e.g., drought) rasters over a period. Default is None.
    arr3 : str, optional
        File path to the aggregated raster file representing the total number of years affected by the third disturbance. Default is None.
    """
    years = years1 + years2
    if years3:
        years = years + years3
    with rio.open(years[0]) as src:
        years_array = src.read(1)
        meta = src.profile
    for year in years[1:]:
        with rio.open(year) as src:
            array = src.read(1)
        years_array += array
        del(array)
        gc.collect()
    years_array[years_array <= 1] = 0
    with rio.open(arr1) as src:
        check1 = src.read(1)
    check1[check1 < 1] = 0
    check1[check1 >= 1] = 1
    years_array = years_array * check1
    del(check1)
    gc.collect()
    with rio.open(arr2) as src:
        check2 = src.read(1)
        meta = src.profile
    check2[check2 < 1] = 0
    check2[check2 >= 1] = 1
    years_array = years_array * check2
    del(check2)
    if arr3:
        with rio.open(arr3) as src:
            check3 = src.read(1)
            meta = src.profile
        check3[check3 < 1] = 0
        check3[check3 >= 1] = 1
        years_array = years_array * check3
        del(check3)
    gc.collect()
    with rio.open(path.join(combo_folder, combo_name), 'w', **meta) as dst:
        dst.write(years_array, 1)

In [ ]:
# Calculate all of the unique combinations with the unique_combos function
five_year_tifs = sorted(glob(path.join(fiveyear_all, "*.tif")))
beetle_tif_list = sorted(glob(path.join(home, "forest-disturbance-stack", "beetle-western-conus", "*.tif")))
fire_tif_list = sorted(glob(path.join(home, "forest-disturbance-stack", "fire-western-conus", "*.tif")))
drought_tif_list = sorted(glob(path.join(home, "forest-disturbance-stack", "drought-western-conus", "*.tif")))
beetle_totals = sorted(glob(path.join(fiveyear_beetle_folder, "beetle_totals_*.tif")))
fire_totals = sorted(glob(path.join(fiveyear_fire_folder, "fire_totals_*.tif")))
drought_totals = sorted(glob(path.join(fiveyear_drought_folder, "drought_totals_*.tif")))
iterable = list(range(len(beetle_tif_list)))

if len(glob(path.join(unique_folder, "drought-beetle", "*.tif"))) == 0:
    for i in iterable:
        if i >=4:
            beetle_years = beetle_tif_list[i-4:i+1]
            fire_years = fire_tif_list[i-4:i+1]
            drought_years = drought_tif_list[i-4:i+1]
            start_year = beetle_years[0][-8:-4]
            end_year = beetle_years[4][-8:-4]
            print("Processing years {} to {}".format(start_year, end_year))
            fire_beetle_output =  "fire_beetle_totals_{}-{}.tif".format(start_year, end_year)
            fire_drought_output = "fire_drought_totals_{}-{}.tif".format(start_year, end_year)
            beetle_drought_output = "bettle_drought_totals_{}-{}.tif".format(start_year, end_year)
            beetle_path = beetle_totals[i-4]
            fire_path = fire_totals[i-4]
            drought_path = drought_totals[i-4]
            print("Combo 1/3")
            unique_combos(beetle_years, fire_years, fire_beetle_out, fire_beetle_output, beetle_path, fire_path)
            print("Combo 2/3")
            unique_combos(fire_years, drought_years, fire_drought_out, fire_drought_output, drought_path, fire_path)
            print("Combo 3/3")
            unique_combos(beetle_years, drought_years, drought_beetle_out, beetle_drought_output, beetle_path, drought_path)

In [ ]:
# Output all data
fire_beetle = sorted(glob(path.join(unique_folder, "fire-beetle", "*.tif")))
fire_drought = sorted(glob(path.join(unique_folder, "fire-dought", "*.tif")))
drought_beetle = sorted(glob(path.join(unique_folder, "drought-beetle", "*.tif")))
fire_beetle_dict = {}
fire_drought_dict = {}
drought_beetle_dict = {}
combos_folders = [fire_beetle, fire_drought, drought_beetle]
combos_dicts = [fire_beetle_dict, fire_drought_dict, drought_beetle_dict]

if not path.isfile(path.join(fire_beetle_out, 'fire_beetle.csv')):
    for i in range(3):
        print("On folder {} of 3".format(i+1))
        dictionary = combos_dicts[i]
        folder = combos_folders[i]
        for file in folder:
            end_year = file[-8:-4]
            with rio.open(file) as src:
                array = src.read(1)
            array[array > 1] == 1
            dictionary[end_year] = (array.sum()* 30 * 30)/1000000

fb_years = list(combos_dicts[0].keys())
fb_area = list(combos_dicts[0].values())
fb_data = {'End Year': fb_years, 'Total Area': fb_area}
fb_plot = pd.DataFrame(fb_data)
fb_plot.to_csv(os.path.join(fire_beetle_out, 'fire_beetle.csv'))
fd_years = list(combos_dicts[1].keys())
fd_area = list(combos_dicts[1].values())
fd_data = {'End Year': fd_years, 'Total Area': fd_area}
fd_plot = pd.DataFrame(fd_data)
fd_plot.to_csv(os.path.join(fire_drought_out, 'fire_drought.csv'))
db_years = list(combos_dicts[2].keys())
db_area = list(combos_dicts[2].values())
db_data = {'End Year': db_years, 'Total Area': db_area}
db_plot = pd.DataFrame(db_data)
db_plot.to_csv(os.path.join(drought_beetle_out, 'drought_beetle.csv'))

In [ ]:
# Find areas where all three disturbance types occured at least once
for i in iterable:
    if i >= 4:
        beetle_years = beetle_tif_list[i-4:i+1]
        fire_years = fire_tif_list[i-4:i+1]
        drought_years = drought_tif_list[i-4:i+1]
        start_year = beetle_years[0][-8:-4]
        end_year = beetle_years[4][-8:-4]
        print("Processing years {} to {}".format(start_year, end_year))
        all_output = "all_unique_totals_{}-{}.tif".format(start_year, end_year)
        beetle_path = beetle_totals[i-4]
        fire_path = fire_totals[i-4]
        drought_path = drought_totals[i-4]
        unique_combos(beetle_years, drought_years, unique_out, all_output, beetle_path, drought_path, fire_years, fire_path)

Processing years 1999 to 2003
Processing years 2000 to 2004
Processing years 2001 to 2005
Processing years 2002 to 2006
Processing years 2003 to 2007
Processing years 2004 to 2008
Processing years 2005 to 2009
Processing years 2006 to 2010
Processing years 2007 to 2011
Processing years 2008 to 2012
Processing years 2009 to 2013
Processing years 2010 to 2014
Processing years 2011 to 2015
Processing years 2012 to 2016
Processing years 2013 to 2017
Processing years 2014 to 2018
Processing years 2015 to 2019
Processing years 2016 to 2020


In [ ]:
unique_dict = {}
folder = sorted(glob(path.join(home, "forest-disturbance-stack", "unique-all", "*.tif")))

for file in folder:
    end_year = file[-8:-4]
    with rio.open(file) as src:
        array = src.read(1)
    array[array > 1] == 1
    unique_dict[end_year] = (array.sum()* 30 * 30)/1000000
unique_dict


{'2003': 150238.9323,
 '2004': 110819.0448,
 '2005': 115620.6078,
 '2006': 140529.7494,
 '2007': 153980.4618,
 '2008': 149573.5731,
 '2009': 159516.9711,
 '2010': 146163.6009,
 '2011': 131321.277,
 '2012': 147399.6204,
 '2013': 144728.6823,
 '2014': 137974.6917,
 '2015': 141409.7172,
 '2016': 133816.6809,
 '2017': 140483.7252,
 '2018': 156511.7919,
 '2019': 157447.1007,
 '2020': 174224.4957}

In [ ]:
years = list(unique_dict.keys())
area = list(unique_dict.values())
data = {'End Year': years, 'Total Area': area}
plot = pd.DataFrame(data)
plot.to_csv(os.path.join(unique_out, 'unique.csv'))